In [6]:

import geopandas
import fiona
import sys
from shapely.geometry import shape, GeometryCollection, Point, MultiPolygon, Polygon, asShape
import numpy as np
import pandas as pd
import shapefile as shp
from shapely import wkt
import ogr
import matplotlib.pyplot as plt
import seaborn as sns



In [7]:
df = pd.read_csv("new_df.csv")
new_df = df.copy()

In [8]:
drv = ogr.GetDriverByName('ESRI Shapefile') #We will load a shape file
ds_in = drv.Open("./deslizamento_de_massas/Movimento_de_Massa_A.shp")    #Get the contents of the shape file
lyr_in = ds_in.GetLayer(0)
geo_ref = lyr_in.GetSpatialRef()
point_ref=ogr.osr.SpatialReference()
point_ref.ImportFromEPSG(4326)
ctran=ogr.osr.CoordinateTransformation(point_ref,geo_ref)

idx_reg = lyr_in.GetLayerDefn().GetFieldIndex("CLASSE")

def check(lon, lat):
    #Transform incoming longitude/latitude to the shapefile's projection
    [lon,lat,z]=ctran.TransformPoint(lon,lat)

    #Create a point
    pt = ogr.Geometry(ogr.wkbPoint)
    pt.SetPoint_2D(0, lon, lat)
    
    #Set up a spatial filter such that the only features we see when we
    #loop through "lyr_in" are those which overlap the point defined above
    lyr_in.SetSpatialFilter(pt)

    #Loop through the overlapped features and display the field of interest
    for feat_in in lyr_in:
        return feat_in.GetFieldAsString(idx_reg)


check(float( -8.056563), float(-35.003045))


'Alta'

In [9]:
counter = 0
is_inside = False;

p = Point( -8.050038,-34.929820)
new_df['danger_level'] =''
for lat, lon in zip(new_df['latitude'], new_df['longitude']):
    label = ''
    lat = float(lat)
    lon = float(lon)
    point = Point(lon, lat)

    label = check(lat, lon)
    
    new_df.at[counter, 'danger_level'] = label
    counter+=1


print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5125 entries, 0 to 5124
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   processo_numero         5125 non-null   object 
 1   solicitacao_data        5125 non-null   object 
 2   solicitacao_hora        5125 non-null   object 
 3   solicitacao_descricao   5125 non-null   object 
 4   solicitacao_bairro      5125 non-null   object 
 5   solicitacao_localidade  5125 non-null   object 
 6   solicitacao_endereco    5125 non-null   object 
 7   processo_ocorrencia     5125 non-null   object 
 8   confirmado              5125 non-null   int64  
 9   latitude                5125 non-null   float64
 10  longitude               5125 non-null   float64
 11  cond                    5125 non-null   bool   
 12  em_recife               5125 non-null   int64  
 13  rain_hour               4855 non-null   float64
 14  rain_day                4850 non-null   

In [10]:
new_df.to_csv(r'C:\Users\parae\Documents\barreiras\barreiras\processing\flooding\danger_level.csv',
            index=False, header=True)

